# W2 - Accessibility (Distance Fields)

In this workshop we will learn the foundations to quantitatively approach spatial accessibility. We will learn about distance fields, construct a euclidean distance field, and construct a manifold distance field.

## 0. Initialization

### 0.1 Importing the packages

In [63]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np

#pv.set_jupyter_backend("ipyvtklink")

# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

### 0.2 import meshes

In [64]:
envelope_path = os.path.relpath('../data/meshes/compulsory_envelope.obj')
context_path = os.path.relpath('../data/meshes/immediate_context.obj')

# load the mesh from file
envelope_mesh = tm.load(envelope_path)
context_mesh = tm.load(context_path)

# Check if the mesh is watertight
print(envelope_mesh.is_watertight)
print(context_mesh.is_watertight)

True
False


### 0.3 Importing the Envelope Lattice

In [65]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/meshes/voxelized_envelope_highres.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

### 0.4 Importing the agent points

In [66]:
# import the streetnetwork as a point cloud
assisted_living= tg.cloud_from_csv("../data/meshes/assisted_living_points.csv")
com_center= tg.cloud_from_csv("../data/meshes/community_center_points.csv")
library= tg.cloud_from_csv("../data/meshes/library_points.csv")
fab_lab= tg.cloud_from_csv("../data/meshes/fab_lab_points.csv")
startups= tg.cloud_from_csv("../data/meshes/startups_points.csv")
shops= tg.cloud_from_csv("../data/meshes/shops_points.csv")
restaurant= tg.cloud_from_csv("../data/meshes/restaurant_points.csv")
lobby= tg.cloud_from_csv("../data/meshes/lobby_points.csv")


In [67]:
# initiating the plotter
p = pv.Plotter()

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# fast visualization of the point cloud

# assisted_living.fast_notebook_vis(p)
# com_center.fast_notebook_vis(p)
# library.fast_notebook_vis(p)
# fab_lab.fast_notebook_vis(p)
# startups.fast_notebook_vis(p)
# shops.fast_notebook_vis(p)
# restaurant.fast_notebook_vis(p)
# lobby.fast_notebook_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(785.7351488880344, 708.3187438880343, 742.5460610261134),
 (65.08283250000001, -12.333572500000002, 21.89374463807907),
 (0.0, 0.0, 1.0)]

In [68]:
# extracting the centroid of all voxels
env_cens = envelope_lattice.centroids_threshold(-1)


### Assisted living

In [69]:
# initializing the distance matrix
dist_assisted = []
# for each voxel ...
for voxel_cen in env_cens:
    # initializing the distance vector (per each voxel)
    dist_assist = []
    # for each street point ...
    for point in assisted_living:
        # find the difference vector
        diff = voxel_cen - point
        # raise the components to the power of two
        diff_p2 = diff**2
        # sum the components
        diff_p2s = diff_p2.sum()
        # compute the square root 
        dist = diff_p2s**0.5
        # add the distance to the distance vector
        dist_assist.append(dist)
    # add the distance vector to the distance matrix
    dist_assisted.append(dist_assist)
# change the distance matrix type, from list to array
dist_assist = np.array(dist_assisted)

# find the distance to the closest street point for each voxel
min_dist_assisted = dist_assist.min(axis=1)
# convert the minimum distance list to a lattice
assisted_eu_distance_lattice = tg.to_lattice(min_dist_assisted.reshape(envelope_lattice.shape), envelope_lattice)
# zero the value of the exterior voxels
assisted = assisted_eu_distance_lattice * envelope_lattice
assisted=1-(assisted-assisted.min())/(assisted.max()-assisted.min())

In [70]:
assisted_living_path = os.path.relpath('../data/fields/assisted_living.csv')
assisted.to_csv(assisted_living_path)

### Library

In [73]:
# initializing the distance matrix
dist_lib = []
# for each voxel ...
for voxel_cen in env_cens:
    # initializing the distance vector (per each voxel)
    dist_v = []
    # for each street point ...
    for point in library:
        # find the difference vector
        diff = voxel_cen - point
        # raise the components to the power of two
        diff_p2 = diff**2
        # sum the components
        diff_p2s = diff_p2.sum()
        # compute the square root 
        dist = diff_p2s**0.5
        # add the distance to the distance vector
        dist_v.append(dist)
    # add the distance vector to the distance matrix
    dist_lib.append(dist_v)
# change the distance matrix type, from list to array
dist_lib = np.array(dist_lib)

# find the distance to the closest street point for each voxel
min_dist_lib = dist_lib.min(axis=1)
# convert the minimum distance list to a lattice
library_eu_distance_lattice = tg.to_lattice(min_dist_lib.reshape(envelope_lattice.shape), envelope_lattice)
# zero the value of the exterior voxels
librar = library_eu_distance_lattice * envelope_lattice
librar=1-(librar-librar.min())/(librar.max()-librar.min())

In [74]:
library_path = os.path.relpath('../data/fields/library.csv')
librar.to_csv(library_path)

### Fab labs

In [75]:
# initializing the distance matrix
dist_fab = []
# for each voxel ...
for voxel_cen in env_cens:
    # initializing the distance vector (per each voxel)
    dist_v = []
    # for each street point ...
    for point in fab_lab:
        # find the difference vector
        diff = voxel_cen - point
        # raise the components to the power of two
        diff_p2 = diff**2
        # sum the components
        diff_p2s = diff_p2.sum()
        # compute the square root 
        dist = diff_p2s**0.5
        # add the distance to the distance vector
        dist_v.append(dist)
    # add the distance vector to the distance matrix
    dist_fab.append(dist_v)
# change the distance matrix type, from list to array
dist_fab = np.array(dist_fab)

# find the distance to the closest street point for each voxel
min_dist_fab = dist_fab.min(axis=1)
# convert the minimum distance list to a lattice
fab_lab_eu_distance_lattice = tg.to_lattice(min_dist_fab.reshape(envelope_lattice.shape), envelope_lattice)
# zero the value of the exterior voxels
fab_labs = fab_lab_eu_distance_lattice * envelope_lattice
fab_labs=1-(fab_labs-fab_labs.min())/(fab_labs.max()-fab_labs.min())

In [76]:
fab_lab_path = os.path.relpath('../data/fields/fab_lab.csv')
fab_labs.to_csv(fab_lab_path)

### Startups

In [77]:
# initializing the distance matrix
dist_start = []
# for each voxel ...
for voxel_cen in env_cens:
    # initializing the distance vector (per each voxel)
    dist_v = []
    # for each street point ...
    for point in startups:
        # find the difference vector
        diff = voxel_cen - point
        # raise the components to the power of two
        diff_p2 = diff**2
        # sum the components
        diff_p2s = diff_p2.sum()
        # compute the square root 
        dist = diff_p2s**0.5
        # add the distance to the distance vector
        dist_v.append(dist)
    # add the distance vector to the distance matrix
    dist_start.append(dist_v)
# change the distance matrix type, from list to array
dist_start = np.array(dist_start)

# find the distance to the closest street point for each voxel
min_dist_start = dist_start.min(axis=1)
# convert the minimum distance list to a lattice
start_eu_distance_lattice = tg.to_lattice(min_dist_start.reshape(envelope_lattice.shape), envelope_lattice)
# zero the value of the exterior voxels
startup = start_eu_distance_lattice * envelope_lattice
startup=1-(startup-startup.min())/(startup.max()-startup.min())


In [78]:
startups_path = os.path.relpath('../data/fields/startups.csv')
startup.to_csv(startups_path)

### Shops

In [79]:
# initializing the distance matrix
dist_shops = []
# for each voxel ...
for voxel_cen in env_cens:
    # initializing the distance vector (per each voxel)
    dist_v = []
    # for each street point ...
    for point in shops:
        # find the difference vector
        diff = voxel_cen - point
        # raise the components to the power of two
        diff_p2 = diff**2
        # sum the components
        diff_p2s = diff_p2.sum()
        # compute the square root 
        dist = diff_p2s**0.5
        # add the distance to the distance vector
        dist_v.append(dist)
    # add the distance vector to the distance matrix
    dist_shops.append(dist_v)
# change the distance matrix type, from list to array
dist_shops = np.array(dist_shops)

# find the distance to the closest street point for each voxel
min_dist_shops = dist_shops.min(axis=1)
# convert the minimum distance list to a lattice
shops_eu_distance_lattice = tg.to_lattice(min_dist_shops.reshape(envelope_lattice.shape), envelope_lattice)
# zero the value of the exterior voxels
shop = shops_eu_distance_lattice * envelope_lattice
shop=1-(shop-shop.min())/(shop.max()-shop.min())


In [80]:
shops_path = os.path.relpath('../data/fields/shops.csv')
shop.to_csv(shops_path)

### Restaurant

In [81]:
# initializing the distance matrix
dist_rest = []
# for each voxel ...
for voxel_cen in env_cens:
    # initializing the distance vector (per each voxel)
    dist_v = []
    # for each street point ...
    for point in restaurant:
        # find the difference vector
        diff = voxel_cen - point
        # raise the components to the power of two
        diff_p2 = diff**2
        # sum the components
        diff_p2s = diff_p2.sum()
        # compute the square root 
        dist = diff_p2s**0.5
        # add the distance to the distance vector
        dist_v.append(dist)
    # add the distance vector to the distance matrix
    dist_rest.append(dist_v)
# change the distance matrix type, from list to array
dist_rest = np.array(dist_rest)

# find the distance to the closest street point for each voxel
min_dist_rest = dist_rest.min(axis=1)
# convert the minimum distance list to a lattice
rest_eu_distance_lattice = tg.to_lattice(min_dist_rest.reshape(envelope_lattice.shape), envelope_lattice)
# zero the value of the exterior voxels
restaur = rest_eu_distance_lattice * envelope_lattice
restaur=1-(restaur-restaur.min())/(restaur.max()-restaur.min())

In [82]:
restaurant_path = os.path.relpath('../data/fields/restaurant.csv')
restaur.to_csv(restaurant_path)

### Lobby

In [83]:
# initializing the distance matrix
dist_lob = []
# for each voxel ...
for voxel_cen in env_cens:
    # initializing the distance vector (per each voxel)
    dist_v = []
    # for each street point ...
    for point in lobby:
        # find the difference vector
        diff = voxel_cen - point
        # raise the components to the power of two
        diff_p2 = diff**2
        # sum the components
        diff_p2s = diff_p2.sum()
        # compute the square root 
        dist = diff_p2s**0.5
        # add the distance to the distance vector
        dist_v.append(dist)
    # add the distance vector to the distance matrix
    dist_lob.append(dist_v)
# change the distance matrix type, from list to array
dist_lob = np.array(dist_lob)

# find the distance to the closest street point for each voxel
min_dist_lob = dist_lob.min(axis=1)
# convert the minimum distance list to a lattice
lob_eu_distance_lattice = tg.to_lattice(min_dist_lob.reshape(envelope_lattice.shape), envelope_lattice)
# zero the value of the exterior voxels
lobb = lob_eu_distance_lattice * envelope_lattice
lobb=1-(lobb-lobb.min())/(lobb.max()-lobb.min())

In [84]:
lobby_path = os.path.relpath('../data/fields/lobby.csv')
lobb.to_csv(lobby_path)

### visualize

In [92]:
l=community

In [93]:
# initiating the plotter
p = pv.Plotter()

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = l.shape
# The bottom left corner of the data set
grid.origin = l.minbound
# These are the cell sizes along each axis
grid.spacing = l.unit

# Add the data values to the cell data
grid.point_arrays["Distance"] = l.flatten(order="F")  # Flatten the Lattice

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
    
# fast visualization of the point cloud
com_center.fast_notebook_vis(p)

# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6]) * 1.5
p.add_volume(grid, cmap="coolwarm", opacity=opacity, shade=True, show_scalar_bar=False)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(785.6075686833789, 708.1911636833788, 743.2184808333789),
 (65.08283250000001, -12.333572500000002, 22.69374465),
 (0.0, 0.0, 1.0)]

## Credits

In [90]:
__author__ = "Shervin Azadi"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/earthy_workshops"
__summary__ = "Earthy Design Studio"